In [41]:
import nltk, re, string, collections
from nltk.util import ngrams
from collections import defaultdict
import orgelpredigt_analysis as oa
import unicodedata

In [145]:
pip install fuzzywuzzy python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [python-Levenshtein]ython-Levenshtein]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#from fuzzywuzzy import fuzz

In [218]:
from rapidfuzz import fuzz

In [16]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/janosch/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [219]:
def fuzzy_match(ngram1, ngram2, threshold=80):
    ngram1_str = ' '.join(ngram1)
    ngram2_str = ' '.join(ngram2)
    similarity_score = fuzz.ratio(ngram1_str, ngram2_str)
    return similarity_score >= threshold

In [142]:
quote = "4. Ich here der Posaunen Thon/Und seh auch den Gerichts Tag schon/ Der mir auch wird ein Urtheil fållen. Hier weiset mein Gewissens-Buch / Da aber des Gesekes Fluch/MichSundenkind hinab zurHöllen/Da/wo man ewig/ewig leidt ! Mord/Jammer/ Angst und Zeter schreyt"

In [122]:


def remove_accents(text):
    # Remove accents except for umlaut characters
    return ''.join(unicodedata.normalize('NFKD', c) if c not in ['ä', 'ö', 'ü', 'Ä', 'Ö', 'Ü'] else c for c in text )

In [141]:
def prepare_text(text):
    text = text.lower()
    text = remove_accents(text)
    text = re.sub(r'[/,;:]', ' ', text)
    return re.sub(r'[^[A-Za-z0-9äöüßÄÖÜæœ\s]','',text)

In [143]:
quote = prepare_text(quote)

In [144]:
quote

'4 ich here der posaunen thon und seh auch den gerichts tag schon  der mir auch wird ein urtheil fallen hier weiset mein gewissensbuch   da aber des gesekes fluch michsundenkind hinab zurhöllen da wo man ewig ewig leidt  mord jammer  angst und zeter schreyt'

In [95]:
id = "E000052"

In [96]:
sermon = oa.Sermon(id)

In [97]:
words = sermon.words

In [98]:
def create_ngrams(text, n):
    tokens = nltk.word_tokenize(text)
    return list(ngrams(tokens, n))

In [99]:
sermon_combined = ' '.join(words)

In [100]:
sermon_combined

'ein wolgerührtes orgelwerck als eine anreitzung zur frucht des geistes hat am xiv sonntage nach trinitatis anno 1721 aus der sonst gewöhnlichen vesper lection an die galater cap v v 16 biß zu ende bey der einweyhung der fürtrefflichen neuen orgel in der kneiphöfischen thumkirchen in öffentlicher predigt vorgestellet und auff verlangen vieler frommer zuhörer dem druck übergeben m christian flottwell diaconus an selbiger kirche und gemeine königsberg gedruckt mit reußnerischen schrifften dem hochedelgebohrnen herrn herrn christoff aegidio negelein seiner königlichen majestät in preussen hochbetrauten geheimbtenund commercienraht wie auch bürgermeister der königlichen stadt kneiphoff denen hochedlen großachtbahren und hochweisen herren stadträhten herrn johann thamm hochansehnlichen pro-consuli und oberkirchenvorsteher herrn christoff leffler hochverdienten voigten herrn christoff krüger hochmeritirenden richter herrn valentin polikein königlichen commercienraht und stadtcamerario herrn 

In [181]:
n = 4
ngrams_sermon = create_ngrams(sermon_combined, n)
ngrams_quote = create_ngrams(quote, n)


In [182]:
ngrams_quote

[('4', 'ich', 'here', 'der'),
 ('ich', 'here', 'der', 'posaunen'),
 ('here', 'der', 'posaunen', 'thon'),
 ('der', 'posaunen', 'thon', 'und'),
 ('posaunen', 'thon', 'und', 'seh'),
 ('thon', 'und', 'seh', 'auch'),
 ('und', 'seh', 'auch', 'den'),
 ('seh', 'auch', 'den', 'gerichts'),
 ('auch', 'den', 'gerichts', 'tag'),
 ('den', 'gerichts', 'tag', 'schon'),
 ('gerichts', 'tag', 'schon', 'der'),
 ('tag', 'schon', 'der', 'mir'),
 ('schon', 'der', 'mir', 'auch'),
 ('der', 'mir', 'auch', 'wird'),
 ('mir', 'auch', 'wird', 'ein'),
 ('auch', 'wird', 'ein', 'urtheil'),
 ('wird', 'ein', 'urtheil', 'fallen'),
 ('ein', 'urtheil', 'fallen', 'hier'),
 ('urtheil', 'fallen', 'hier', 'weiset'),
 ('fallen', 'hier', 'weiset', 'mein'),
 ('hier', 'weiset', 'mein', 'gewissensbuch'),
 ('weiset', 'mein', 'gewissensbuch', 'da'),
 ('mein', 'gewissensbuch', 'da', 'aber'),
 ('gewissensbuch', 'da', 'aber', 'des'),
 ('da', 'aber', 'des', 'gesekes'),
 ('aber', 'des', 'gesekes', 'fluch'),
 ('des', 'gesekes', 'fluch', 'm

In [183]:
def search_ngrams(needle, haystack):
    matches = []
    for ngram in haystack:
        if ngram in needle:
            matches.append(ngram)
    return matches

In [247]:
def search_ngrams_fuzzy(needle, haystack, threshold=80):
    matches = []
    for i in range(len(haystack)):
        for ngram_a in needle:
            if fuzzy_match(ngram_a, haystack[i], threshold):
                matches.append(haystack[i])
                break  # Stop searching once a match is found
    return matches

In [178]:
def fuzzy_match(needle, haystack, threshold=80):
    needle_str = ' '.join(needle)
    haystack_str = ' '.join(haystack)
    sim_score = fuzz.ratio(needle_str, haystack_str)
    return sim_score >= threshold

In [167]:
def search_ngrams_combined(needle,haystack,threshold=80):
    matches_exact = search_ngrams(needle, haystack)
    if len(matches_exact) > 0:
        return matches_exact
    else:
        return search_ngrams_fuzzy(needle, haystack, threshold)
        


In [211]:
from concurrent.futures import ThreadPoolExecutor

In [ ]:
import marisa_trie

In [172]:
def build_trie(ngrams):
    ngram_strings = [' '.join(ngram) for ngram in ngrams]
    return marisa_trie.Trie(ngram_strings)

In [173]:
def search_trie(trie, ngram):
    ngram_str = ' '.join(ngram)
    return ngram_str in trie

In [179]:
def search_ngrams_with_trie(needle, haystack, threshold=80):
    matches = []
    trie_a = build_trie(needle)

    for ngram_b in haystack:
        ngram_b_str = ' '.join(ngram_b)
        if ngram_b_str in trie_a:
            matches.append(ngram_b)
        else:
            for ngram_a in trie_a.keys():
                ngram_a_tuple = tuple(ngram_a.split())
                if fuzzy_match(ngram_a_tuple, ngram_b, threshold):
                    matches.append(ngram_b)
                    break
    return matches

In [185]:
found_ngrams = search_ngrams(ngrams_quote, ngrams_sermon)
found_ngrams

[]

In [186]:
found_ngrams_fuzzy = search_ngrams_fuzzy(ngrams_quote, ngrams_sermon)
ngrams_to_text(found_ngrams_fuzzy)

'ich höre der posaunenthon und sehe den gerichtstag schon ach'

In [227]:
with open('source_texts/praxis_pietatis_melica/1057.txt', 'r') as f:
    page = f.read()

In [228]:
page = prepare_text(page)
page

'sterbegesange 1015 in die posaune stoffen so sol dann seel und leib zugleich mit dir in deines vaters reich zu deiner freud eingehen 10 ists nun dein will so stell dich ein mich selig zu verseken ach ewig bey und mit dir seyn wie hoch muß das ergohen eröffne dich du todes pfort auf daß an solchen schönen ort jch durch dich mige fahren paul gerb 948 bin ja herr in deiner macht du hast mich an das licht gebracht du unterhältst mir auch das leben du kennest meiner monden zahl weist wann ich diesem jammerthal auch wieder gute kacht muß geben wo wie und wan ich sterben so das weift du vater mehr als wol 2 wen hab ich nun als dich allein der mir in meiner lekten pein mit trost und rath weiß beyzu springen  wer nimmt sich meiner seelen an wann nun mein leben nichts mehr kan und ich muß mit dem tode ringen wann aller sinnen krafft ge bricht  thuft dues gott mein heyland nicht  3 mich dunckt da lieg ich schon vor mir in grof fer hitz ohn krafft und zier mit höchster herkens angst befallen gehö

In [229]:
ngrams_page = create_ngrams(page, n)

In [232]:
ngrams_sermon_copy = ngrams_sermon

In [248]:
found_ngrams = search_ngrams_fuzzy(ngrams_page, ngrams_sermon, 80)
found_ngrams

[('ich', 'höre', 'der', 'posaunenthon', 'und'),
 ('der', 'posaunenthon', 'und', 'sehe', 'den'),
 ('sehe', 'den', 'gerichtstag', 'schon', 'ach')]

In [241]:
def search_sermon_for_songquotes(ngrams_pages, ngrams_sermon, threshold=80):
    matches = []
    for page in ngrams_pages:
        for i in range(len(ngrams_sermon)):
            for ngram_a in page:
                if fuzzy_match(ngram_a, ngrams_sermon[i], threshold):
                    print(ngrams_sermon[i])
                    matches.append(ngrams_sermon[i])
                    #ngrams_sermon.pop(i)
                    break  # Stop searching once a match is found
    return matches

In [242]:
search_sermon_for_songquotes([page], ngrams_sermon_copy, 90)

[]

In [194]:
import json

In [200]:
known_song_quotes = list(set([q for q in sermon.all_references if q.startswith('E10')]))
print(known_song_quotes)
with open('songs_to_pages_mapping.json') as f:
    songbook_pages = json.load(f)

relevant_pages = []
for i in known_song_quotes:
    page = songbook_pages[i]['pages']
    if page != '':
        page = str(int(page) + 42)          # add 42 to offset prefatory matter in scans
        add_page = str(int(page) + 1)
        relevant_pages.append([i, page])
        relevant_pages.append([i, add_page])
print(relevant_pages)

['E100022', 'E100140', 'E100139', 'E100138', 'E100057', 'E100132', 'E100141', 'E100078', 'E100134', 'E100067', 'E100017']
[['E100022', '740'], ['E100022', '741'], ['E100138', '1057'], ['E100138', '1058'], ['E100057', '608'], ['E100057', '609'], ['E100132', '226'], ['E100132', '227'], ['E100141', '650'], ['E100141', '651'], ['E100078', '642'], ['E100078', '643'], ['E100134', '1039'], ['E100134', '1040'], ['E100067', '925'], ['E100067', '926'], ['E100017', '434'], ['E100017', '435']]


In [201]:
def ngrams_to_text(ngrams):
    text = []
    for i in range(0, len(ngrams)):
        if i == 0:
            text += list(ngrams[i])
        else:
            text.append(list(ngrams[i])[-1])
    return ' '.join(text)


# Searching for quotes in known pages with exact 4-gram matches

In [206]:
total_quotes = 0

for id, pagenr in relevant_pages:
    with open(f'source_texts/praxis_pietatis_melica/{pagenr}.txt', 'r') as f:
        page = f.read()
    page = prepare_text(page)
    ngrams_page = create_ngrams(page, n)
    found_ngrams = search_ngrams(ngrams_page, ngrams_sermon)
    #found_ngrams = search_ngrams_with_trie(ngrams_page, ngrams_sermon,90)
    if len(found_ngrams) > 0:
        print(f"=== Found N-Grams on page {pagenr} ===")
        print(f"Source: {oa.get_short_info(id)}")
        print(f'"{ngrams_to_text(found_ngrams)}"')
        total_quotes += 1
    else:
        print(f"=== No N-Grams found on page {pagenr} ===")

print(f"Total number of quotes of at least {n} words found: {total_quotes}")

=== No N-Grams found on page 740 ===
=== Found N-Grams on page 741 ===
Source: Nicolai, Philipp: Wie schön leuchtet der Morgenstern
"in steter liebe wallen"
=== No N-Grams found on page 1057 ===
=== No N-Grams found on page 1058 ===
=== Found N-Grams on page 608 ===
Source: anonym: Herr Jesu Christ, dich zu uns wend
"mund zum lobe dein mehr stärck den verstand daß uns dein"
=== No N-Grams found on page 609 ===
=== No N-Grams found on page 226 ===
=== Found N-Grams on page 227 ===
Source: N.N.: Treuer Gott ich muß dir klagen
"in den streit meinen glauben täglich mehr damit ich den feind kan schlagen alle pfeile von mir jagen"
=== No N-Grams found on page 650 ===
=== Found N-Grams on page 651 ===
Source: N.N.: Herr, dir trau ich all mein Tage
"soll dir allzeit bringen meine zunge"
=== Found N-Grams on page 642 ===
Source: N.N.: Herr Gott, dich loben wir
"zahl und die lieben propheten all die theuren märtrer schall"
=== No N-Grams found on page 643 ===
=== No N-Grams found on page 1039 ==

# Searching for quotes in known pages with fuzzy search

In [225]:
total_quotes = 0
n = 5

ngrams_sermon = create_ngrams(sermon_combined, n)

for id, pagenr in relevant_pages:
    with open(f'source_texts/praxis_pietatis_melica/{pagenr}.txt', 'r') as f:
        page = f.read()
    page = prepare_text(page)
    ngrams_page = create_ngrams(page, n)
    #found_ngrams = search_ngrams(ngrams_page, ngrams_sermon)
    found_ngrams = search_ngrams_fuzzy(ngrams_page, ngrams_sermon,90)
    if len(found_ngrams) > 0:
        print(f"=== Found N-Grams on page {pagenr} ===")
        print(f"Source: {oa.get_short_info(id)}")
        print(f'"{ngrams_to_text(found_ngrams)}"')
        total_quotes += 1
    else:
        print(f"=== No N-Grams found on page {pagenr} ===")

print(f"Total number of quotes of at least {n} words found: {total_quotes}")

=== No N-Grams found on page 740 ===
=== Found N-Grams on page 741 ===
Source: Nicolai, Philipp: Wie schön leuchtet der Morgenstern
"in cithara und last die süsse erschallen daß ich möge liebe wallen"
=== No N-Grams found on page 1057 ===
=== No N-Grams found on page 1058 ===
=== Found N-Grams on page 608 ===
Source: anonym: Herr Jesu Christ, dich zu uns wend
"fein den glauben mehr stärck den verstand daß uns dein nahm"
=== No N-Grams found on page 609 ===
=== No N-Grams found on page 226 ===
=== Found N-Grams on page 227 ===
Source: N.N.: Treuer Gott ich muß dir klagen
"so offt ich muß in den streit meinen glauben täglich mehr damit ich den feind kan schlagen alle pfeile von mir jagen"
=== No N-Grams found on page 650 ===
=== Found N-Grams on page 651 ===
Source: N.N.: Herr, dir trau ich all mein Tage
"mein hertz soll dir allzeit bringen deines meine zunge singen jch"
=== Found N-Grams on page 642 ===
Source: N.N.: Herr Gott, dich loben wir
"der heiligen zwölff bohten zahl und prophet

# Searching for quotes in all pages with exact 4-gram matches

In [217]:
total_quotes = 0
n = 4

for i in range(42, 1290):
    with open(f'source_texts/praxis_pietatis_melica/{i}.txt', 'r') as f:
        page = f.read()
    page = prepare_text(page)
    ngrams_page = create_ngrams(page, n)
    found_ngrams = search_ngrams(ngrams_page, ngrams_sermon)
    #found_ngrams = search_ngrams_with_trie(ngrams_page, ngrams_sermon,90)
    if len(found_ngrams) > 0:
        print(f"=== Found N-Grams on page {i} ===")
        print(f'"{ngrams_to_text(found_ngrams)}"')
        total_quotes += 1

print(f"Total number of quotes of at least {n} words found: {total_quotes}")

=== Found N-Grams on page 43 ===
"uns zeitlich und ewig"
=== Found N-Grams on page 59 ===
"sey gott in der höhe"
=== Found N-Grams on page 73 ===
"gott vater sohn und"
=== Found N-Grams on page 83 ===
"gott vater sohn und"
=== Found N-Grams on page 150 ===
"gott vater sohn und"
=== Found N-Grams on page 212 ===
"und wonne hören daß die"
=== Found N-Grams on page 219 ===
"gott vater sohn und"
=== Found N-Grams on page 227 ===
"in den streit meinen glauben täglich mehr damit ich den feind kan schlagen alle pfeile von mir jagen"
=== Found N-Grams on page 271 ===
"wie der wind die"
=== Found N-Grams on page 407 ===
"da die engel singen klingen"
=== Found N-Grams on page 416 ===
"gott in der höhe"
=== Found N-Grams on page 424 ===
"weg von mir das"
=== Found N-Grams on page 425 ===
"gott der herr und"
=== Found N-Grams on page 435 ===
"mehr denn da da die engel die schellen klingen in regis curia eya"
=== Found N-Grams on page 564 ===
"in acht zu nehmen"
=== Found N-Grams on page 608 ===
"m

# Searching for quotes in all pages with fuzzy 5-gram matches

In [226]:
total_quotes = 0
n = 5

ngrams_sermon = create_ngrams(sermon_combined, n)


for i in range(42, 1290):
    with open(f'source_texts/praxis_pietatis_melica/{i}.txt', 'r') as f:
        page = f.read()
    page = prepare_text(page)
    ngrams_page = create_ngrams(page, n)
    #found_ngrams = search_ngrams(ngrams_page, ngrams_sermon)
    found_ngrams = search_ngrams_fuzzy(ngrams_page, ngrams_sermon,90)
    if len(found_ngrams) > 0:
        print(f"=== Found N-Grams on page {i} ===")
        print(f'"{ngrams_to_text(found_ngrams)}"')
        total_quotes += 1

print(f"Total number of quotes of at least {n} words found: {total_quotes}")

=== Found N-Grams on page 44 ===
"und sie mit freuden zu"
=== Found N-Grams on page 59 ===
"sey gott in der höhe"
=== Found N-Grams on page 63 ===
"vater sohn und heiligen geist"
=== Found N-Grams on page 73 ===
"gott vater sohn und heiligen"
=== Found N-Grams on page 83 ===
"gott vater sohn und heiligen geist"
=== Found N-Grams on page 122 ===
"daß wir nicht alle in"
=== Found N-Grams on page 137 ===
"und die eurigen mit seiner"
=== Found N-Grams on page 150 ===
"gott vater sohn und heiligen geist"
=== Found N-Grams on page 173 ===
"den teuffel und die welt"


KeyboardInterrupt: 